In [1]:
from __future__ import print_function 
import numpy as np
import tensorflow as tf 

import os
import sys

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Lambda
from keras.layers import Conv1D, MaxPooling1D, Embedding

import keras.backend as K

Using TensorFlow backend.


## Load the dataset

In [2]:
f = np.load('data_and_embedding.npz')

In [3]:
num_words = f['num_words']
embedding_dim = f['embedding_dim']
max_sequence_length = f['max_sequence_length']

data = f['data']
labels = f['labels']

embedding_matrix = f['embedding_matrix']

In [4]:
validation_split = 0.2 
epoch = 10

In [5]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(validation_split * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

## Classification model (Keras)

### Build the model 

In [6]:
embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_sequence_length,
                            trainable=False)

In [7]:
def embedding_mean(x):
    return tf.reduce_mean(x, axis=1)

In [8]:
sequence_input = Input(shape=(max_sequence_length, ), dtype='int32')
# print(sequence_input.shape)
embedded_sequences = embedding_layer(sequence_input)
# print(embedded_sequences.shape)
x = Lambda(embedding_mean)(embedded_sequences)
# print(x.shape)
preds = Dense(6, activation='softmax')(x)
# print(preds.shape)

model = Model(sequence_input, preds)

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, array(1000))       0         
_________________________________________________________________
embedding_1 (Embedding)      (None, array(1000), array 2000000   
_________________________________________________________________
lambda_1 (Lambda)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 606       
Total params: 2,000,606.0
Trainable params: 606.0
Non-trainable params: 2,000,000.0
_________________________________________________________________


### Train the model

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
import time 
start_time = time.time()

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

print("Training time: ", time.time() - start_time)

Train on 311902 samples, validate on 77975 samples
Epoch 1/10
 45824/311902 [===>..........................] - ETA: 75s - loss: 1.5094 - acc: 0.6404 

### Save the model

In [ ]:
model.save('models/logistic_regression.h5')